In [ ]:
import os
import zipfile 
import gdown
from tqdm import tqdm




## Fetch data from Google Drive 
# Root directory for the dataset
data_root = 'dataset'
# Path to folder with the dataset
dataset_folder = f'{data_root}/img_align_celeba'
# URL for the CelebA dataset
url1 = 'https://drive.google.com/uc?id=0B7EVK8r0v71pZjFTYXZWM3FlRnM'
url2 = 'https://drive.google.com/uc?id=1mcqEu1vN1BUQ8IkEqXd3f54eaVxQsLIO'
# Path to download the dataset to
download_path1 = f'{data_root}/img_align_celeba.zip'
download_path2 = f'{data_root}/list_attr_celeba.csv'
# Create required directories 
if not os.path.exists(data_root):
  os.makedirs(data_root)
  os.makedirs(dataset_folder)

# Download the dataset from google drive
if not os.path.exists(download_path1):
  gdown.download(url1, download_path1, quiet=False)
if not os.path.exists(download_path2):
  gdown.download(url2, download_path2, quiet=False)


with zipfile.ZipFile(download_path1, 'r') as ziphandler:
    for file in tqdm(ziphandler.namelist(), desc='Extraction du fichier'):
        ziphandler.extract(file, dataset_folder)

## Test Preprocess


In [ ]:
!python PreProcess.py --root-images "./dataset/img_align_celeba/img_align_celeba" --root-attributes "./dataset/list_attr_celeba.csv"
# ou lance directement la commande dans le terminal << python PreProcess.py --root-images "./dataset/img_align_celeba/img_align_celeba" --root-attributes "./dataset/list_attr_celeba.csv" >>

## Test data.py

In [ ]:
from data import Datasets


dataset = Datasets(root_images="archive/img_align_celeba/img_align_celeba/",root_atributes="archive/list_attr_celeba.csv")

In [ ]:
data_image, attributes = dataset[0]

In [ ]:
data_image.shape

In [ ]:
from skimage import io

# Afficher l'image
io.imshow(data_image.permute(1, 2, 0).numpy())

# Attendre que l'utilisateur ferme la fenêtre
io.show()